In [18]:
import os
import subprocess
#os.chdir('OrthologyRelationships')  # Executer une seule fois puisque la seconde fois on se trouve deja dans le bon repertoire 
filelist=[i for i in os.listdir()] # 0 1-ToMany  1 paralogy 2 1to1  3 ToMany

In [2]:
asc=['Phmamm','Phfumi','Cisavi','Cirobu','Moocci','Moocul','Mooccu','Boschl','Boleac','Haaura','Harore']
vert=['Cmil','Lcha','Hsap','Mmus','Ggal','Psin']
outgroupt=['Spur','Apla','Bbel','Blan']

In [3]:
def getasc(file): # On va piocher tout les gene d'ascidies donc ceux appartenants 
    ascilist=[]
    max=100
    with open (file) as f : 
        lines= f.readlines()
        for line in lines : 
            if max!=0 : 
                for gene in line.split(' '): 
                    if len(gene.split('|')[1].split('.'))>2: 
                        ascilist.append(gene.strip())
                        #if not(gene.split('|')[0] in asc) :         # verfication effectué et valider on a bien que des ascidies 
                            #print(gene)                 
    f.close()
    return list(set(ascilist))      # On passe par un set pour prendre les valeurs uniques de la liste
ascilist=getasc(filelist[2])
miniasci=ascilist[:100]  # mini jeu pour les tests et essais 
moyenasc=ascilist[:10000]

In [4]:
len(ascilist)

94661

In [1]:
def grep(Gene,file): 
    commande='grep '+Gene.split('|')[1]+' '+file
    #print(commande)
    res=subprocess.check_output(commande.split(' '))
    return res.decode().replace("\n",' ').split(' ')[:-1]   # dernière element à enlever car toujours un espaces blanc 

In [6]:
def withhuman(Gene,file):   # Return True si le gene a un ortologue chez l'humain, False sinon 
    res=grep(Gene,file)
    for i in res : 
        if i.split('|')[0]=='Hsap':
            return True
    return False


In [7]:
len(ascilist)/len(miniasci)*8/60/60  #temps estimer en heures pour l'execution totale 

2.1035777777777778

In [8]:
def dicrelation(GeneList,file):
    dic={}
    for g in GeneList : 
        l=list(set(grep(g,file)))
        l.remove(g)
        dic[g]=l
    return dic 

In [158]:
dicrel=dicrelation(miniasci,filelist[0])

CalledProcessError: Command '['grep', 'Boschl.CG.Botznik2013.chr9.g50001', 'rootedTre_orthology1ToMany.txt']' returned non-zero exit status 1.

In [10]:
def withhumanbis(Gene):   # Return True si le gene a un ortologue chez l'humain, False sinon 
    res=dicrel[Gene]
    for i in res : 
        if i.split('|')[0]=='Hsap':
            return True
    return False

#Resultats ttest H0 tps=sans grep ~ tpsbis = avec grep H1 tps significativement différent , T= 24.9, pvalue= 6e-63

In [11]:
import sys 
# cout en memoire du dic actuelle et cout en memoire estimé du dic complet 
print(sys.getsizeof(dicrel)/1000) # en Ko
print((len(ascilist)/len(dicrel)*sys.getsizeof(dicrel))/1000) # en Ko

4.696
4445.28056


In [165]:
def gethumans(Gene,dicrel): 
    res=dicrel[Gene]
    r=[]
    for relation in res : 
        if relation.split('|')[0]=='Hsap':
            r.append(relation)
    return r


In [13]:
vertnh=vert
vertnh.remove('Hsap')

In [166]:
def getvert(Gene,dicrel): 
    res=dicrel[Gene]
    d={}
    rel=[]
    for relation in res : 
        if relation.split('|')[0] in vertnh:
            rel.append(relation)
    d[Gene]=rel
    return d

In [162]:
def writehuman(GeneList,dicrel): 
    os.chdir('../Outputs')
    with open('human.txt','w') as f : 
        for g in GeneList : 
           if gethumans(g,dicrel)!=[]: 
                for rel in gethumans(g,dicrel): 
                    f.write(g+' '+rel)
                    print(rel)
                    try : 
                        res=formateout(rel.split('|')[0],rel.split('|')[1].split('.')[0])[rel.split('|')[1].split('.')[0]]
                        for name in res.keys(): 
                              f.write('_Name::'+name)
                              for synid in res[name].keys(): 
                                  print(synid)
                                  f.write(';_Syn'+synid)
                                  for syn in res[name][synid]: 
                                      f.write('::'+syn )
                        f.write("\n")
                    except subprocess.CalledProcessError :
                        f.write('_Name::notFound'+"\n") 
                        print('Warning '+rel.split('|')[1].split('.')[0]+' not found in '+rel.split('|')[0])
    os.chdir('../OrthologyRelationships')

In [163]:
def writevert(GeneList,dicrel): 
    os.chdir('../Outputs')
    with open('vertnh.txt','w') as f : 
        for g in GeneList : 
            if getvert(g,dicrel)!=[]: 
                for rel in getvert(g,dicrel)[g]: 
                    f.write(g+' '+rel)
                    print(rel)
                    try : 
                        res=formateout(rel.split('|')[0],rel.split('|')[1].split('.')[0])[rel.split('|')[1].split('.')[0]]
                        for name in res.keys(): 
                              f.write('_Name::'+name)
                              for synid in res[name].keys(): 
                                  print(synid)
                                  f.write(';_Syn'+synid)
                                  for syn in res[name][synid]: 
                                      f.write('::'+syn )
                        f.write("\n")
                    except subprocess.CalledProcessError :
                        f.write('_Name::notFound'+"\n") 
                        print('Warning '+rel.split('|')[1].split('.')[0]+' not found in '+rel.split('|')[0])
    os.chdir('../OrthologyRelationships')

In [157]:
#dicrel=dicrelation(moyenasc,filelist[2])
writehuman(miniasci)
#writevert(miniasci)


Hsap|ENSG00000145041.15
.17
Hsap|ENSG00000173465.7
.8
Hsap|ENSG00000074201.8
.9
Hsap|ENSG00000099800.7
.8
Hsap|ENSG00000101843.18
.19
Hsap|ENSG00000213339.8
.9
Hsap|ENSG00000133104.12
.14
Hsap|ENSG00000007545.15
.16
Hsap|ENSG00000003056.7
.8
Hsap|ENSG00000165609.12
.14
Hsap|ENSG00000167088.10
.11
Hsap|ENSG00000132763.14
.15
Hsap|ENSG00000269964.2
.4
Hsap|ENSG00000099917.17
.17
Hsap|ENSG00000132554.19
.21
Hsap|ENSG00000106344.8
.8
Hsap|ENSG00000032389.12
.13
Hsap|ENSG00000049167.13
.16
Hsap|ENSG00000112379.8
.9
Hsap|ENSG00000164117.12
.14
Hsap|ENSG00000156414.18
.20
Hsap|ENSG00000156973.13
.14
Hsap|ENSG00000123219.12
.13
Hsap|ENSG00000162735.18
.19
Hsap|ENSG00000087087.18
.20
Hsap|ENSG00000116903.7
.7
Hsap|ENSG00000036672.15
.16
Hsap|ENSG00000225830.10
.16


In [13]:

def findsymb(espece,assembl): 
    os.chdir('../Ensembl')
    commande='grep '+assembl+' '+espece+'.csv'
    res=subprocess.check_output(commande.split(' '))
    os.chdir('../OrthologyRelationships')
    return res.decode().split("\n")[:-1]   # dernière element à enlever car toujours un espaces blanc 


In [94]:
def formateout(espece,assembl): 
    dic={}
    for res in findsymb(espece,assembl): 
        if res.split(',')[0] in dic.keys(): 
            if res.split(',')[3]=='': 
                new=['None']
            else : 
                new=[res.split(',')[3]]
                
            if '.'+res.split(',')[1].split('.')[1] in sousdic[res.split(',')[2]].keys():
                 sousdic[res.split(',')[2]]['.'+res.split(',')[1].split('.')[1]].append(new[0])
            else : 
                sousdic[res.split(',')[2]]['.'+res.split(',')[1].split('.')[1]]=new
        else : 
            sousdic={}
            syn={}
            if res.split(',')[3]=='': 
                syn['.'+res.split(',')[1].split('.')[1]]=['None']
            else : 
                syn['.'+res.split(',')[1].split('.')[1]]=[res.split(',')[3]]
            sousdic[res.split(',')[2]]=syn
        dic[assembl]=sousdic
    return dic

In [159]:
def Nametunicates(file,limit): 
    ascs=getasc(file)[:limit]
    drel=dicrelation(ascs,file)
    writehuman(ascs,drel)
    writevert(ascs,drel)
    

In [58]:
findsymb('Ggal','ENSGALG0000')

['ENSGALG00010002543,ENSGALG00001002543.1,UCP3,UCP']

In [63]:

def Pouletgrep(Gene,file): 
    commande='grep '+Gene.split('|')[1]+' '+file
    #print(commande)
    res=subprocess.check_output(commande.split(' ')).decode().split("\n")[:-1]
    tab=[]
    for line in res : 
        if line.split("ID=")[1].split(':')[0]=='gene': 
            l=line.split("ID=gene:")[1].split(';')[0]
            l=l+','+line.split("ID=gene:")[1].split(';')[0]+'.'+line.split("version=")[1].split(';')[0]
            l=l+','+line.split("Name=")[1].split(';')[0]+','
            tab.append(l)
    return tab

Pouletgrep('Ggal|ENSGALG0000002073','../Ensembl/Ggal.gff3')

CalledProcessError: Command '['grep', 'ENSGALG00000012073', '../Ensembl/Ggal.gff3']' returned non-zero exit status 1.

In [67]:
Ordervert=['Hsap','Mmus','Psin','Ggal','Lcha','Cmil']
def hierarchise(file): 
    with open(file) as f, open('hier_'+file.split('/')[len(file.split('/'))-1],'w') as hier: 
        lines=f.readlines()
        current=''
        relist=[]
        for line in lines: 
            if line.split(' ')[0]!=current : 
                current=line.split(' ')[0]
                if len(relist)>=1 and relist[0]!=' ': 
                    k=0 
                    stop=0
                    while k<len(Ordervert) and stop==0: 
                        #print(relist)
                        for rel in relist: 
                            if rel.split(' ')[1].split('|')[0]==Ordervert[k]: 
                                #print(rel)
                                stop=1
                                hier.write(rel)
                        k+=1
                relist=[]
            relist.append(line)

In [70]:
hierarchise('Outputs/manyToMany_all.txt')